In [19]:
#1.Imports and Reproducibility
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# Fix seeds
torch.manual_seed(42)
np.random.seed(42)

In [20]:
#2.Custom Dataset (Using New Binary CSV)
class HAM500Binary(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.df = pd.read_csv(csv_path)  # standard comma-separated CSV
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]['image_id']
        label = int(self.df.iloc[idx]['binary_label'])  # already 0 or 1

        # Try common image extensions
        for ext in ['.jpg', '.png', '.jpeg']:
            path = os.path.join(self.img_dir, img_id + ext)
            if os.path.exists(path):
                image = Image.open(path).convert('RGB')
                break
        else:
            raise FileNotFoundError(f"Image not found: {img_id}")

        if self.transform:
            image = self.transform(image)
        return image, label

In [21]:
#3.Data Loading and 80/20 Split

# Define transform (64x64, normalized)
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset using new binary CSV
dataset = HAM500Binary(
    csv_path='../HAM5000/HAM500_metadata_binary.csv',
    img_dir='../HAM5000/HAM500_images',
    transform=transform
)


# Fixed 80/20 split with seed
indices = torch.randperm(len(dataset)).tolist()
n_train = int(0.8 * len(dataset))
train_indices = indices[:n_train]
val_indices = indices[n_train:]

train_set = Subset(dataset, train_indices)
val_set = Subset(dataset, val_indices)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

print(f"Train: {len(train_set)} samples | Val: {len(val_set)} samples")

Train: 400 samples | Val: 100 samples


In [22]:
# Full dataset statistics
all_labels = [dataset[i][1] for i in range(len(dataset))]
n_malade_total = sum(all_labels)
n_saine_total = len(all_labels) - n_malade_total
print("=== FULL DATASET ===")
print(f"Total samples: {len(dataset)}")
print(f"Malade (mel): {n_malade_total} ({n_malade_total / len(dataset) * 100:.1f}%)")
print(f"Saine (others): {n_saine_total} ({n_saine_total / len(dataset) * 100:.1f}%)\n")

# Training set statistics
train_labels = [dataset[i][1] for i in train_indices]
n_malade_train = sum(train_labels)
n_saine_train = len(train_labels) - n_malade_train
print("=== TRAINING SET ===")
print(f"Total samples: {len(train_set)}")
print(f"Malade (mel): {n_malade_train} ({n_malade_train / len(train_set) * 100:.1f}%)")
print(f"Saine (others): {n_saine_train} ({n_saine_train / len(train_set) * 100:.1f}%)\n")

# Validation set statistics
val_labels = [dataset[i][1] for i in val_indices]
n_malade_val = sum(val_labels)
n_saine_val = len(val_labels) - n_malade_val
print("=== VALIDATION SET ===")
print(f"Total samples: {len(val_set)}")
print(f"Malade (mel): {n_malade_val} ({n_malade_val / len(val_set) * 100:.1f}%)")
print(f"Saine (others): {n_saine_val} ({n_saine_val / len(val_set) * 100:.1f}%)")

=== FULL DATASET ===
Total samples: 500
Malade (mel): 43 (8.6%)
Saine (others): 457 (91.4%)

=== TRAINING SET ===
Total samples: 400
Malade (mel): 35 (8.8%)
Saine (others): 365 (91.2%)

=== VALIDATION SET ===
Total samples: 100
Malade (mel): 8 (8.0%)
Saine (others): 92 (92.0%)


In [23]:
#4.Common Classification Head (VGG16 Head)
class VGG16Head(nn.Module):
    def __init__(self, input_dim=512, num_classes=2):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_dim, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        return self.fc3(x)

In [24]:
#5. Model 1 – Custom CNN (From Scratch)
class CustomCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            # Block 2
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            # Block 3
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            # Block 4
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            # Block 5: 1x1 conv + GAP
            nn.Conv2d(256, 512, 1), nn.ReLU(),
        )
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.head = VGG16Head(input_dim=512)

    def forward(self, x):
        x = self.features(x)
        x = self.gap(x).view(x.size(0), -1)
        return self.head(x)

In [25]:
#6. Model 2 – VGG16 (From Scratch)
class VGG16(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.head = VGG16Head(input_dim=512)

    def forward(self, x):
        x = self.features(x)
        x = self.gap(x).view(x.size(0), -1)
        return self.head(x)

In [26]:
#7. Models 3 & 4 – Vision Transformer (ViT-1 and ViT-2)
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=64, patch_size=8, in_chans=3, embed_dim=64):
        super().__init__()
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.n_patches = (img_size // patch_size) ** 2

    def forward(self, x):
        x = self.proj(x)
        x = x.flatten(2).transpose(1, 2)
        return x

class ViTEncoderBlock(nn.Module):
    def __init__(self, embed_dim=64, n_heads=4, mlp_ratio=2, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, n_heads, dropout=dropout, batch_first=True)
        self.norm2 = nn.LayerNorm(embed_dim)
        hidden_dim = embed_dim * mlp_ratio
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, embed_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        attn_out, _ = self.attn(self.norm1(x), self.norm1(x), self.norm1(x))
        x = x + attn_out
        x = x + self.mlp(self.norm2(x))
        return x

class ViT(nn.Module):
    def __init__(self, n_blocks=1, num_classes=2):
        super().__init__()
        self.patch_embed = PatchEmbedding(embed_dim=64)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, 64))
        self.pos_embed = nn.Parameter(torch.zeros(1, 65, 64))  # 64 patches + 1 cls token
        self.encoder = nn.Sequential(*[ViTEncoderBlock() for _ in range(n_blocks)])
        self.to_512 = nn.Linear(64, 512)
        self.head = VGG16Head(input_dim=512)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat([cls_tokens, x], dim=1)
        x = x + self.pos_embed
        x = self.encoder(x)
        features = self.to_512(x[:, 0])  # Use [CLS] token
        return self.head(features)

In [27]:
#8. Training and Evaluation Function
def train_and_evaluate(model, train_loader, val_loader, epochs=8, lr=1e-3, model_name="Model"):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    history = {'train_loss': [], 'val_loss': [], 'val_acc': [], 'val_f1': []}

    for epoch in range(epochs):
        # Training
        model.train()
        total_train_loss = 0.0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        history['train_loss'].append(total_train_loss / len(train_loader))

        # Validation
        model.eval()
        total_val_loss = 0.0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Metrics (binary classification)
        all_labels = np.array(all_labels, dtype=int)
        all_preds = np.array(all_preds, dtype=int)
        acc = accuracy_score(all_labels, all_preds)
        prec = precision_score(all_labels, all_preds, zero_division=0, average='binary')
        rec = recall_score(all_labels, all_preds, zero_division=0, average='binary')
        f1 = f1_score(all_labels, all_preds, zero_division=0, average='binary')

        history['val_loss'].append(total_val_loss / len(val_loader))
        history['val_acc'].append(acc)
        history['val_f1'].append(f1)

        print(f"[{model_name}] Epoch {epoch+1}/{epochs} | Acc: {acc:.4f} | Recall (Malade): {rec:.4f} | F1: {f1:.4f}")

    # Save curves
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.title(f'{model_name} - Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history['val_acc'], label='Accuracy')
    plt.plot(history['val_f1'], label='F1-score')
    plt.title(f'{model_name} - Metrics')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'courbes_{model_name.replace(" ", "_")}.png')
    plt.close()

    return {'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1}

In [28]:
#9. Train All Four Models

models = {
    "CustomCNN": CustomCNN(),
    "VGG16": VGG16(),
    "ViT-1": ViT(n_blocks=1),
    "ViT-2": ViT(n_blocks=2)
}

results = {}
for name, model in models.items():
    print(f"\n--- Training {name} ---")
    results[name] = train_and_evaluate(model, train_loader, val_loader, epochs=8, model_name=name)


--- Training CustomCNN ---
[CustomCNN] Epoch 1/8 | Acc: 0.9200 | Recall (Malade): 0.0000 | F1: 0.0000
[CustomCNN] Epoch 2/8 | Acc: 0.9200 | Recall (Malade): 0.0000 | F1: 0.0000
[CustomCNN] Epoch 3/8 | Acc: 0.9200 | Recall (Malade): 0.0000 | F1: 0.0000
[CustomCNN] Epoch 4/8 | Acc: 0.9200 | Recall (Malade): 0.0000 | F1: 0.0000
[CustomCNN] Epoch 5/8 | Acc: 0.9200 | Recall (Malade): 0.0000 | F1: 0.0000
[CustomCNN] Epoch 6/8 | Acc: 0.9200 | Recall (Malade): 0.0000 | F1: 0.0000
[CustomCNN] Epoch 7/8 | Acc: 0.9200 | Recall (Malade): 0.0000 | F1: 0.0000
[CustomCNN] Epoch 8/8 | Acc: 0.9200 | Recall (Malade): 0.0000 | F1: 0.0000

--- Training VGG16 ---
[VGG16] Epoch 1/8 | Acc: 0.9200 | Recall (Malade): 0.0000 | F1: 0.0000
[VGG16] Epoch 2/8 | Acc: 0.9200 | Recall (Malade): 0.0000 | F1: 0.0000
[VGG16] Epoch 3/8 | Acc: 0.9200 | Recall (Malade): 0.0000 | F1: 0.0000
[VGG16] Epoch 4/8 | Acc: 0.9200 | Recall (Malade): 0.0000 | F1: 0.0000
[VGG16] Epoch 5/8 | Acc: 0.9200 | Recall (Malade): 0.0000 | F1: 

In [29]:
#10: Print Final Results Table (For Word Report)

print("\nFINAL VALIDATION METRICS (AFTER 8 EPOCHS):")
print("-" * 75)
print(f"{'Model':<12} | {'Accuracy':<10} | {'Precision':<10} | {'Recall (Malade)':<15} | {'F1-score':<10}")
print("-" * 75)
for name, metrics in results.items():
    print(f"{name:<12} | {metrics['accuracy']:<10.4f} | {metrics['precision']:<10.4f} | {metrics['recall']:<15.4f} | {metrics['f1']:<10.4f}")


FINAL VALIDATION METRICS (AFTER 8 EPOCHS):
---------------------------------------------------------------------------
Model        | Accuracy   | Precision  | Recall (Malade) | F1-score  
---------------------------------------------------------------------------
CustomCNN    | 0.9200     | 0.0000     | 0.0000          | 0.0000    
VGG16        | 0.9200     | 0.0000     | 0.0000          | 0.0000    
ViT-1        | 0.9200     | 0.0000     | 0.0000          | 0.0000    
ViT-2        | 0.9200     | 0.0000     | 0.0000          | 0.0000    


In [30]:
# Sauvegarder chaque modèle après entraînement

import os
os.makedirs('models', exist_ok=True)

for name, model in models.items():
    torch.save(model.state_dict(), f"models/{name}.pth")
    print(f"\n--- the model '{name}' saved in 'models/{name}.pth' successfully ---")




--- the model 'CustomCNN' saved in 'models/CustomCNN.pth' successfully ---

--- the model 'VGG16' saved in 'models/VGG16.pth' successfully ---

--- the model 'ViT-1' saved in 'models/ViT-1.pth' successfully ---

--- the model 'ViT-2' saved in 'models/ViT-2.pth' successfully ---
